Write a PyTorch implementation of a Variational Autoencoder (VAE) for the MNIST dataset with a 2D latent space. Please ensure that the following requirements are satisfied:

- Scale and normalize the dataset
- Division of the training dataset into training and validation sets.
- A convolutional neural network (CNN) for both the encoder and decoder.
- A Gaussian distribution for q(z|x) in the encoder
- A bernoulli distribution for p(x|z) in the decoder using a sigmoid
- The loss function is defined as a separate method, adjusted for the distribution used for the decoder.
- Use of GPU or MPS if available.

In [ ]:
Here's a PyTorch implementation of a Variational Autoencoder (VAE) for the MNIST dataset with a 2D latent space, incorporating all of your specified requirements:

```python
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Check for GPU or MPS availability
device = torch.device('cuda' if torch.cuda.is_available() else
                      'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using device: {device}')

# Load and preprocess the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Scales images to [0,1]
    # Note: We avoid normalization to keep pixel values between 0 and 1 for Bernoulli distribution
])

# Load the dataset and split into training and validation sets
full_train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)

# Split the training data into training and validation sets (80% training, 20% validation)
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),  # Output: [batch_size, 32, 14, 14]
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),  # Output: [batch_size, 64, 7, 7]
            nn.ReLU(),
        )
        self.fc1 = nn.Linear(64*7*7, 256)
        self.fc_mu = nn.Linear(256, 2)      # Mean of latent space
        self.fc_logvar = nn.Linear(256, 2)  # Log variance of latent space

        # Decoder
        self.decoder_input = nn.Linear(2, 256)
        self.fc2 = nn.Linear(256, 64*7*7)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),  # Output: [batch_size, 32, 14, 14]
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),  # Output: [batch_size, 16, 28, 28]
            nn.ReLU(),
            nn.Conv2d(16, 1, kernel_size=3, padding=1),  # Output: [batch_size, 1, 28, 28]
            nn.Sigmoid()  # For Bernoulli distribution
        )
    
    def encode(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)  # Standard deviation
        eps = torch.randn_like(std)  # Sample from standard normal
        return mu + eps * std        # Reparameterization trick

    def decode(self, z):
        x = torch.relu(self.decoder_input(z))
        x = torch.relu(self.fc2(x))
        x = x.view(-1, 64, 7, 7)
        x = self.decoder(x)
        return x
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        return x_hat, mu, logvar

# Define the loss function
def loss_function(recon_x, x, mu, logvar):
    # Binary cross-entropy loss for Bernoulli distribution
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    # Kullback-Leibler divergence between q(z|x) and p(z)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Initialize the model and optimizer
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
epochs = 10

for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0
    for data, _ in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    
    average_train_loss = train_loss / len(train_loader.dataset)
    print(f'Epoch {epoch}, Average Training Loss: {average_train_loss:.4f}')

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, _ in val_loader:
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            loss = loss_function(recon_batch, data, mu, logvar)
            val_loss += loss.item()
    average_val_loss = val_loss / len(val_loader.dataset)
    print(f'Epoch {epoch}, Average Validation Loss: {average_val_loss:.4f}')

# After training, you can test the model, generate samples, or save the model as needed.
```

**Explanation of Key Components:**

- **Scaling and Normalizing the Dataset:**
  - Used `transforms.ToTensor()` to scale images to the [0, 1] range.
  - Skipped normalization to keep pixel values between 0 and 1, suitable for Bernoulli distribution.

- **Division of Training and Validation Sets:**
  - Split the full training dataset into 80% training and 20% validation using `random_split`.

- **Convolutional Neural Networks for Encoder and Decoder:**
  - **Encoder:**
    - Two convolutional layers with ReLU activation.
    - Outputs the mean and log variance (`mu` and `logvar`) for the Gaussian distribution.
  - **Decoder:**
    - Starts with the latent vector `z` after reparameterization.
    - Uses transpose convolutional layers to reconstruct the image.
    - Final activation is `Sigmoid` to output probabilities for Bernoulli distribution.

- **Gaussian Distribution for q(z|x) in the Encoder:**
  - The encoder outputs `mu` and `logvar`, parameters of the Gaussian distribution.

- **Bernoulli Distribution for p(x|z) in the Decoder:**
  - The decoder's final activation function is `Sigmoid`, providing outputs suitable for Bernoulli distribution.

- **Loss Function Defined Separately:**
  - The `loss_function` method calculates the binary cross-entropy loss and Kullback-Leibler divergence.

- **Use of GPU or MPS if Available:**
  - The code checks for CUDA (GPU) or MPS support and uses it if available.
  - Data and model are moved to the appropriate device.

**Note:**
- This implementation is suitable for educational purposes and may need hyperparameter tuning or architecture adjustments for production use.
- After training, you can visualize the reconstructed images or latent space representations to evaluate the VAE's performance.
```

This implementation includes detailed comments and explanations to help understand each component. It satisfies all the requirements you've specified.